In [1]:
import numpy as np
import pandas as pd
import random
import time
import csv
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import itertools
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import nltk
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import sumy
from rouge import Rouge
from readability import Readability

In [21]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.summarizers.lsa import LsaSummarizer

In [26]:
input_path='F:/Course/2021 Fall/ISE 540/Project/y_eva' # evaluation directory
output_path='F:/Course/2021 Fall/ISE 540/Project/y_eva/eva_result.csv' # export csv result
book_list=os.listdir(input_path)                

In [ ]:
df_eva=pd.DataFrame(columns=['book','Summary_type','rougle_1_r','rougle_1_p','rougle_1_f','rougle_2_r','rougle_2_p','rougle_2_f','rougle_l_r','rougle_l_p','rougle_l_f','cos_sim','f_score','f_ease','f_grade_levels'])
done=[]
wrong=[]
for i in range(0,len(book_list),2):
    try:
        def read_article(file_name):
            file = open(file_name, "r", encoding="UTF-8")
            filedata = file.readlines()
            # print(filedata)

            filedata_new = " ".join(filedata)               # join every elements in the list into a single element
            paragraph = filedata_new.replace("\n \n ", ".") # replace newlines between paragraphs by periods(there are usually 2 or 3 newlines between paragraphs, so replace by at least 2 newlines)
            paragraph = paragraph.split(".")                # Split all periods, so we get sentence by sentence
            # print(paragraph)

            sentences = []
            for sentence in paragraph:
                sentence = sentence.replace("\n", "") # remove the extra newlines between sentences
                # print(sentence)
                sentences.append(sentence.replace("[^a-zA-Z]", " ").split(" "))

            sentences = [[string for string in sublist if string] for sublist in sentences] # Remove the empty elements in every sentences if any
            sentences = [string for string in sentences if string] # remove empty lists if any
            # sentences.pop() # Drop the last two empty elements
            end=0
            for test in sentences:
                if ('***END' in test) or (('END' in test)&('***' in test)):
                    break
                end=end+1

            if end!=len(sentences):
                del sentences[end:]

            start=0
            for test in sentences:
                if ('***START' in test )or (('START' in test)&('***' in test)):
                    break
                start=start+1
            if start!=len(sentences): 
                del sentences[:start+1]

            return sentences

        book=read_article(input_path+'/'+book_list[i] )  # reading the book from directory
        
        cur_book=book_list[i]                           # record the book index  
        
        eva = open(input_path+'/'+book_list[i]+'_eva.txt', "r", encoding="UTF-8")    #read corresponding evaluation
        eva = eva.readlines()
        eva=" ".join(eva)
        eva=eva.replace('\n','').replace("\'","").replace('--','. ').replace(' — ','. ').replace('.. ','. ')
        eva=re.sub('\s+',' ',eva)

        def split(input_book):          # chapterize the book

            cnt=0
            idx=dict()

            for i in input_book:
                if ('chapter' in map(str.lower, i[:1])) & (len(i)>=2) :
                    idx[tuple(i)]=cnt

                elif ('chapitre' in map(str.lower, i[:1])) & (len(i)>=2) :
                    idx[tuple(i)]=cnt   

                elif ('chap' in map(str.lower, i[:1])):
                    idx[tuple(i)]=cnt

                elif ('section' in map(str.lower, i[:1])):
                    idx[tuple(i)]=cnt    

                cnt=cnt+1

            page=sorted(idx.values())

            chapter=dict()
            cnt=0
            while cnt <=len(page)-1:

                if cnt!=len(page)-1:
                    chapter[cnt+1]=input_book[page[cnt]:page[cnt+1]]

                else:
                    chapter[cnt+1]=input_book[page[cnt]:]
                cnt=cnt+1
            return chapter

        chap=split(book)

        if chap =={}: # backup chapterize method if split function fail
            point=1
            for page in range(300,len(book),300):
                if (page+300)>len(book):
                    chap[point]=book[page-300:page]
                    chap[point+1]=book[page:]
                else:
                    chap[point]=book[page-300:page]
                point=point+1
                
                
                
                
        def chap_string(input_para): # convert chap into string
            string=[]
            for tok in input_para:
                string.append(" ".join(tok))
            string='. '.join(string)
            return(string)
        
        def agg_sum(input_sum): # convert individual output sentences from Sumy into one-paragraph summary
            in_sum=[]
            for sens in input_sum:
                in_sum.append(str(sens))
            in_sum=' '.join(in_sum)
            return(in_sum)
        
        def cos_sim(result_sum,evaluation): # calculate cosine similarity
            cos_data=[result_sum, evaluation]
            vectorizer_c = TfidfVectorizer( )
            vector = vectorizer_c.fit_transform(cos_data)

            sim_rs=cosine_similarity(vector)[1][0]
            return sim_rs


        tot_sum=[]
        top_1=[]

        for i in chap:



            chap_sum=[]
            for j in chap[i]:
                chap_sum.append(" ".join(j))
            chap_sum=' '.join(chap_sum) 

            data=[chap_sum] #record tf-idf info
            vectorizer = TfidfVectorizer( )
            vector1 = vectorizer.fit_transform(data)
            tf_sum=sum(vector1.toarray()[0]) 

            tf_len=len(vector1.toarray()[0]) 


            parser = PlaintextParser.from_string(chap_string(chap[i]),Tokenizer("english"))
            summarizer = LexRankSummarizer()
            summary = summarizer(parser.document,4)

            try:
                top_1.append(agg_sum([summary[0]]))
            except:
                pass

            tot_sum.append(agg_sum(summary))
        tot_sum=' '.join(tot_sum)
        top_1=' '.join(top_1)


        r=Readability(eva)
        f = r.flesch()
        df= {'book':cur_book,'Summary_type':'Evaluation','rougle_1_r':'NA','rougle_1_p':'NA','rougle_1_f':'NA','rougle_2_r':'NA','rougle_2_p':'NA','rougle_2_f':'NA','rougle_l_r':'NA','rougle_l_p':'NA','rougle_l_f':'NA','cos_sim':'NA','f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
        df_eva=df_eva.append(df, ignore_index = True)
        
        
        # sometimes baseline model might fail to meet the requirement to calculate rouge score or flesh score, when error raise
        # NA will be input as that value
        
        #choose top1 sentence
        rouge = Rouge()

        try:
            res=rouge.get_scores(top_1, eva)[0]
            cur_sim=cos_sim(top_1, eva)
            r=Readability(top_1)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'Top1_Sent','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)
        except:
            res=rouge.get_scores(top_1, eva)[0]    
            cur_sim=cos_sim(top_1, eva)


            df= {'book':cur_book,'Summary_type':'Top1_Sent','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':'NA','f_ease':'NA','f_grade_levels':'NA'}
            df_eva=df_eva.append(df, ignore_index = True)        
        
        
        #choose first 5 sentence
        base_nst=[]
        for i in chap:
            for j in chap[i][:5]:
                base_nst.append(agg_sum(j))
        base_nst=' '.join(base_nst)
        
        try:
            res=rouge.get_scores(base_nst, eva)[0]
            cur_sim=cos_sim(base_nst, eva)
            r=Readability(base_nst)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'first_n_sents','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)
        except:

            cur_sim=cos_sim(base_nst, eva)
            r=Readability(base_nst)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'first_n_sents','rougle_1_r':'NA','rougle_1_p':'NA','rougle_1_f':'NA','rougle_2_r':'NA','rougle_2_p':'NA','rougle_2_f':'NA','rougle_l_r':'NA','rougle_l_p':'NA','rougle_l_f':'NA','cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)

        #choose last 5 sentence
        base_last=[]
        for i in chap:
            for j in chap[i][-5:]:
                base_last.append(agg_sum(j))
        base_last=' '.join(base_last)
        
        try:
            res=rouge.get_scores(base_last, eva)[0]
            cur_sim=cos_sim(base_last, eva)
            r=Readability(base_last)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'last_n_sents','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)
        except:

            cur_sim=cos_sim(base_last, eva)
            r=Readability(base_last)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'last_n_sents','rougle_1_r':'NA','rougle_1_p':'NA','rougle_1_f':'NA','rougle_2_r':'NA','rougle_2_p':'NA','rougle_2_f':'NA','rougle_l_r':'NA','rougle_l_p':'NA','rougle_l_f':'NA','cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)

        #choose n random sentence
        n=3
        base_rand=[]

        for i in chap:
            rand_choice=random.sample(range(0,len(chap[i])),n)
            for j in rand_choice:
                base_rand.append(agg_sum(chap[i][j]))
        base_rand=' '.join(base_rand)
        try:
            res=rouge.get_scores(base_rand, eva)[0]
            cur_sim=cos_sim(base_rand, eva)
            r=Readability(base_rand)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'rand_n_sents','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)
        except:

            cur_sim=cos_sim(base_rand, eva)
            r=Readability(base_rand)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'rand_n_sents','rougle_1_r':'NA','rougle_1_p':'NA','rougle_1_f':'NA','rougle_2_r':'NA','rougle_2_p':'NA','rougle_2_f':'NA','rougle_l_r':'NA','rougle_l_p':'NA','rougle_l_f':'NA','cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)

        #choose 3 longest sentence
        n=3
        base_long=[]

        for i in chap:
    
            sens_long=sorted(chap[i],key=lambda x:-len(x))[:n]
            for j in sens_long:
                base_long.append(agg_sum(j))



        base_long=' '.join(base_long)

        try:
            res=rouge.get_scores(base_long, eva)[0]
            cur_sim=cos_sim(base_long, eva)
            r=Readability(base_long)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'longest_n_sents','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)
        except:

            cur_sim=cos_sim(base_long, eva)
            r=Readability(base_long)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'longest_n_sents','rougle_1_r':'NA','rougle_1_p':'NA','rougle_1_f':'NA','rougle_2_r':'NA','rougle_2_p':'NA','rougle_2_f':'NA','rougle_l_r':'NA','rougle_l_p':'NA','rougle_l_f':'NA','cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)

        #choose 10 shorted sentence
        n=10
        base_short=[]

        for i in chap:
            longest=1
            sens_long=sorted(chap[i],key=lambda x:len(x))[:n]
            for j in sens_long:
                base_short.append(agg_sum(j))



        base_short=' '.join(base_short)

        try:
            res=rouge.get_scores(base_short, eva)[0]
            cur_sim=cos_sim(base_short, eva)
            r=Readability(base_short)
            f = r.flesch()
            df= {'book':cur_book,'Summary_type':'shortest_n_sents','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
            df_eva=df_eva.append(df, ignore_index = True)
        except:
            res=rouge.get_scores(base_short, eva)[0]    
            cur_sim=cos_sim(base_short, eva)


            df= {'book':cur_book,'Summary_type':'shortest_n_sents','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':'NA','f_ease':'NA','f_grade_levels':'NA'}
            df_eva=df_eva.append(df, ignore_index = True)
            
        #model summary
        res=rouge.get_scores(tot_sum, eva)[0]
        cur_sim=cos_sim(tot_sum, eva)
        r=Readability(tot_sum)
        f = r.flesch()
        df= {'book':cur_book,'Summary_type':'model_sum','rougle_1_r':res['rouge-1']['r'],'rougle_1_p':res['rouge-1']['p'],'rougle_1_f':res['rouge-1']['f'],'rougle_2_r':res['rouge-2']['r'],'rougle_2_p':res['rouge-2']['p'],'rougle_2_f':res['rouge-2']['p'],'rougle_l_r':res['rouge-l']['r'],'rougle_l_p':res['rouge-l']['p'],'rougle_l_f':res['rouge-l']['f'],'cos_sim':cur_sim,'f_score':f.score,'f_ease':f.ease,'f_grade_levels':f.grade_levels}
        df_eva=df_eva.append(df, ignore_index = True)
        done.append(cur_book)
        df_eva.to_csv(output_path,index=False)
        
        print(done)
    except:
        wrong.append(cur_book)
        continue